<a href="https://colab.research.google.com/github/josephgonz12/MAS4115/blob/main/vector_space_model_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reminder about vector spaces
... and intro to the **vector space model** for texts!

> All this is practical, and is inspired by my work on https://www.google.com/alerts . More elaborate techniques are used nowadays, but the basic principles are the same.

The idea is to map each short text document into a vector.

> Reminder, vector is just an element of a vector space, in our case  $\mathbb{R}^d$.

> $\mathbb{R}^d$ means $\underbrace{\mathbb{R} \times \mathbb{R} ... \times \mathbb{R}}_{d \ \text{times}}$, where $\times$ means the Cartesian product. For example if  $v \in \mathbb{R}^2$, then it's a pair $(a,b)$, such that $a \in \mathbb{R}$ and also $b \in \mathbb{R}$. For example $(1.5, 2.0)$.



Using a fixed list of $10$ keywords, we count the number occurences
of each keyword in a document. we form the documents **term vector** whose i-th
element is the count of the i-th keyword in this document.

> In practice, there could be millions or more keywords... So we'd work with $\mathbb{R}^{10^6}$ for example, although in this case the vectors would be stored in a sparse format, so only the (few) nonzero entries would take up memory.

Since there are $10$ keywords, each document becomes a vector in $\mathbb{R}^{10}$, namely the space of 10-dimensional  vectors.

> Remember that the dimension of a vector is just the number of its elements. So 10 dimensional vectors have 10 positions in which you can put a real number. In this number is just a nonnegative integer, that's okay. A 10-dimensional vector space is just a vector space in which 10-dimensional vectors live.

Once we have the term vectors, we can -- for example -- compute distance between them. So **searching for the most similar document** (given some **query term-vector**) boils down to looking for the (geometrically) nearest vector!

> You can think of this as a simple version of google. You have a query e.g. "cats playing with dogs youtube", which is converted to a term-vector (see below) and the best-matching document (text in a website, textual description of a video) is returned. Below we play with an example.

> Warning: the approach given below can be improved significantly, with
an easy trick. So try to be critical about what we do!

In [1]:
import numpy as np

keywords = ["cats", "dogs", "trees", "fields", "binary",
            "photosynthesis", "whiskers", "animals",
            "holes", "nature"]

# some text below <- BTW. I'm a comment, not part of the Python program, hi!

In [ ]:
print(keywords)

['cats', 'dogs', 'trees', 'fields', 'binary', 'photosynthesis', 'whiskers', 'animals', 'holes', 'nature']


In [2]:
# @title
docs = [
    """The purring of a cat is often associated with contentment,
but it can also indicate pain relief and self-healing. Cats
groom themselves meticulously using their tongues, a behavior
that helps regulate body temperature and establish a sense
of ownership over their territory.""",

    """Dogs, descendants of wolves, have been bred for various
roles, from herding livestock to providing companionship and
support. Their remarkable diversity in size and appearance
is a result of selective breeding, shaping them into
breeds optimized for specific tasks and environments.
Dogs exhibit a strong sense of loyalty and attachment
to their human companions, forming deep bonds of
affection.""",

    """Deciduous trees shed their leaves in the fall as an
adaptation to conserve water and energy during colder months.
Leaves contain specialized cells called chloroplasts that
use sunlight to convert carbon dioxide and water into
energy-rich carbohydrates, a process known as photosynthesis.
This natural phenomenon is a remarkable example of
ecological adaptation.""",

    """In computer science, a binary tree is a hierarchical
data structure with nodes that have at most two children,
commonly used in search algorithms. Binary trees can be
balanced or unbalanced, impacting the efficiency of
search operations. The concept of binary trees extends
to various applications, including the representation
of hierarchical data in databases and the construction
of efficient data compression algorithms.""",

 """The purring of a cat is often associated with contentment,
but it can also indicate pain relief and self-healing. Cats
groom themselves meticulously using their tongues, a behavior
that helps regulate body temperature and establish a sense
of ownership over their territory.
The purring of a cat is often associated with contentment,
but it can also indicate pain relief and self-healing. Cats
groom themselves meticulously using their tongues, a behavior
that helps regulate body temperature and establish a sense
of ownership over their territory.
The purring of a cat is often associated with contentment,
but it can also indicate pain relief and self-healing. Cats
groom themselves meticulously using their tongues, a behavior
that helps regulate body temperature and establish a sense
of ownership over their territory.""",

    """Black holes, predicted by Einstein's theory of
general relativity, have such strong gravitational effects
that nothing, not even light, can escape their grasp.
These enigmatic cosmic objects form when massive stars
undergo gravitational collapse. The intense gravitational
forces near a black hole create a phenomenon known as
time dilation, where time passes more slowly for an
observer near the event horizon than for those further
away.""",

    """Cat whiskers are highly sensitive tactile hairs that
help cats detect changes in their surroundings and navigate
in the dark. These whiskers are embedded deep within the
skin and are connected to nerve endings, allowing cats
to perceive even subtle air movements. Whiskers play
a crucial role in a cat's spatial awareness and ability
to judge whether they can fit through tight spaces.""",

    """Dogs' exceptional sense of smell has been utilized in
tasks like tracking scents, detecting medical conditions,
and even identifying certain cancers. A dog's olfactory
system is incredibly intricate, featuring hundreds of
millions of scent receptors that can detect minute
amounts of odor molecules. This heightened sense of
smell makes dogs invaluable in various fields, from
search and rescue operations to medical diagnostics.""",

    """Ancient cultures revered sacred trees as symbols of
life and growth, often attributing spiritual significance
to their towering presence. Trees like the oak, with its
strong and resilient wood, were often associated with
strength and protection. The symbolism of trees varies
across cultures, but their role as providers of shelter,
food, and resources has remained a constant throughout
human history.""",

    """A spanning tree of a graph is a subgraph that includes
all the vertices and is also a tree, crucial in network
design and optimization. Spanning trees are employed
in various computer science applications, such as
establishing efficient communication networks and
designing fault-tolerant systems. The concept of
spanning trees extends beyond the realm of graphs and
has implications in both computer science and engineering.""",

    """Worm holes, hypothetical shortcuts in spacetime, captivate
the imagination of physicists as potential portals for
interstellar travel. These theoretical constructs
are akin to tunnels connecting distant points in space,
potentially allowing for faster-than-light travel. The
concept of worm holes emerged from the equations of
general relativity, but their existence and viability
remain speculative topics in the realm of theoretical
physics.""",

    """Known for their loyalty, dogs have been used as
service animals for tasks like guiding the visually impaired
and assisting individuals with disabilities. Service dogs
undergo rigorous training to perform specific tasks,
such as guiding individuals through obstacles and alerting
them to potential dangers. These highly trained dogs
enhance the quality of life for people with various
disabilities, fostering greater independence and
confidence.""",

    """The growth rings of trees, visible in cross-sections
of trunks, hold records of environmental changes and can
provide insights into historical climates. Each ring
represents a year of growth, with alternating patterns
of light and dark reflecting variations in growth
conditions. Dendrochronology, the study of tree rings,
enables scientists to reconstruct past climate patterns,
understand ecological shifts, and even date historical
artifacts and structures.""",

    """In graph theory, a rooted tree designates one node
as the root and assigns a parent-child relationship, useful
in representing hierarchical structures. Trees in graph
theory find applications in diverse fields, including
database organization, computer network topologies,
and evolutionary biology. The tree structure simplifies
the representation and manipulation of complex relationships,
making it a fundamental concept in various domains.""",

    """Sink holes, formed by the collapse of underground cavities,
can emerge suddenly and pose significant challenges to urban
planning. These geological phenomena occur when soluble rock
layers dissolve over time, leaving voids that can cause the
ground above to collapse. Sink holes can range in size from
small depressions to massive craters, and their occurrence
has prompted the development of strategies to detect,
prevent, and mitigate their impact.""",

    """Cat whiskers are highly sensitive tactile hairs that
help cats detect changes in their surroundings and navigate
in the dark. These specialized hairs, known as vibrissae,
contain sensory nerve endings that allow cats to sense
objects and obstacles without relying solely on their
vision. Vibrissae are found on various parts of a cat's
body, including the face, paws, and even the backs of
their forelegs.""",

    """Dogs' barking, ranging from alert barks to play barks,
is a complex form of communication that expresses various
emotions and intentions. Dogs use barking to convey
excitement, fear, warning, or even to solicit attention.
The tone, duration, and frequency of barks can vary
significantly, and dogs often modify their barking
patterns based on their surroundings and interactions
with humans and other animals.""",

    """Cats are known for their independent nature, while dogs are known for their loyalty and companionship.
Cats are self-sufficient and content alone, while dogs thrive on social interaction and attention.
Cats groom themselves meticulously using whiskers, while dogs have a remarkable sense of smell for various tasks.
Both animals bring unique qualities to their owners.
""",

    """Decision trees are a predictive modeling tool that
recursively divides data into subsets based on features,
used extensively in machine learning. Each internal node
represents a decision based on a specific feature, guiding
the data down different branches until reaching a leaf node,
which represents the predicted outcome. Decision trees
provide transparency in decision-making processes and
find applications in various fields, from medical diagnosis
to financial forecasting"""
]

In [3]:
# No need to understand these, we'll just use them.

def get_documents_term_list(doc):
  '''
  Gets all the documents' terms in order, after some basic cleanup.
  '''
  things_to_strip = ".,;`'\"-_!?*+"
  return [res for x in doc.split() if (res:= x.strip(things_to_strip).lower())]


def get_characteristic_words(corpus):
  '''
  Here we just cheat a little and get a predefined list!
  '''
  # return sorted(set.union(*map(get_terms, corpus)))
  return keywords


def get_term_dictionary(corpus):
  '''
  Returns a mapping from a term (string) to its unique index.
  '''
  terms = get_characteristic_words(corpus)
  # no need to understand it... for now
  return dict(zip(terms, range(len(terms))))


def get_documents_term_vector(doc, term_dict):
  '''
  Creates the term-vector of doc using terms
  in term_dict.
  '''
  term_vector = np.zeros(len(term_dict))
  for term in get_documents_term_list(doc):
    if term in term_dict:
      term_index = term_dict[term]
      term_vector[term_index] += 1
  return term_vector


def get_term_vector_matrix(corpus):
  '''
  Returns a matrix of size num_documents x num_characteristic_words
  The i-th row corresponds to the term-vector of the i-th document.
  '''
  term_dict = get_term_dictionary(corpus)
  N = len(term_dict)
  matrix = np.zeros((len(corpus), N))
  for i, doc in enumerate(docs):
    tv = get_documents_term_vector(doc, term_dict)
    matrix[i] = tv
  return matrix


def get_human_readable_term_vector(tv, keywords):
  '''
  Just in case you wanted to print a term vector nicely.
  '''
  return [(keywords[i], float(cnt)) for i, cnt in enumerate(tv) if cnt > 0]

You can check the term vectors of some of the documents below:

In [4]:
term_dict = get_term_dictionary(docs)
print(term_dict)

{'cats': 0, 'dogs': 1, 'trees': 2, 'fields': 3, 'binary': 4, 'photosynthesis': 5, 'whiskers': 6, 'animals': 7, 'holes': 8, 'nature': 9}


In [5]:
which_doc = 2
print("document snippet:", docs[which_doc][:80])
tv = get_documents_term_vector(docs[which_doc], term_dict)
print(tv)

document snippet: Deciduous trees shed their leaves in the fall as an
adaptation to conserve water
[0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]


In [6]:
get_human_readable_term_vector(tv, keywords)

[('trees', 1.0), ('photosynthesis', 1.0)]

Lets build the big matrix describing all our documents (aka our corpus).

In [7]:
M = get_term_vector_matrix(docs)
print(M)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 2. 0. 3. 0. 0. 0. 0. 0.]
 [3. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [2. 0. 0. 0. 0. 0. 3. 0. 0. 0.]
 [0. 2. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 3. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0.]
 [0. 3. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0.]
 [2. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 3. 0. 0. 0. 0. 0. 1. 0. 0.]
 [3. 3. 0. 0. 0. 0. 1. 1. 0. 1.]
 [0. 0. 2. 1. 0. 0. 0. 0. 0. 0.]]


# Operations
One nice thing about vectors is that we can add and subtract them, and multiply by a scalar (real number) -- and get another vector as a result.

> More technically if $v,w \in \mathbb{R}^d$ then $v+w \in \mathbb{R}^d$, $v-w \in \mathbb{R}^d$ and $cv \in \mathbb{R}^d$ for any $c \in \mathbb{R}$.

What do these operations mean in our case? E.g. What does it mean to add two term-vectors. Or multiply one by, say 3. Or -2.

> Let's discuss!

# Part II: Distances and dissimilarities

## Euclidean distance
The Euclidean distance between two vectors $p,q$ in $\mathbb{R}^d$ is defined as $\sqrt{\sum_{i=1}^d (p_i-q_i)^2 }$

> It's similar to computing the hypothenuse of a right triangle in two dimensions: $\sqrt{x^2 +y^2}$

This distance comes from the Euclidean norm (aka $L2$ norm), which measures the distance of a vector from the *origin* (the 0 vector), or in other words the magnitude (length) of a vector $v$: $\|v\|_2 = \sqrt{\sum_{i=1}^{d} v_i^2}$. The distance between $p,q$ is just $\|p-q\|_2$.

> That's why we'll often use the np.linalg.norm, which (by default) computes this $L2$ norm of a given vector.

In [9]:
def eucl_distance(v, w):
  v = np.asarray(v)
  return np.linalg.norm(v-w)

# Let's play with this!

#Task 1:

Choose indices of term vectors in the matrix, $M$, compute the distance/dissimilarity between them, and check how it works. **Do pairs of vectors with smaller distance really indeed correspond to similar documents?**

Also:
* What does multiplying a term vector by a number correspond to?  

* How does multiplying a vector by a positive constant affect the dissimilarity (to some other vector)?

* What does the norm of a term vector tell us?

In [10]:
doc_index_1 = 4 # change this
doc_index_2 = 6 # ... and this!
print(eucl_distance(M[doc_index_1], M[doc_index_2]))

3.1622776601683795


In [11]:
print(f'Start of first doc: "{docs[doc_index_1][:150]} [...]"')
print(f'Start of second doc: "{docs[doc_index_2][:150]} [...]"')
print("Term-vectors of these documents: ")
print(M[[doc_index_1, doc_index_2]])

print("human readable versions:")
print(get_human_readable_term_vector(M[doc_index_1], keywords=keywords))
print(get_human_readable_term_vector(M[doc_index_2], keywords=keywords))

Start of first doc: "The purring of a cat is often associated with contentment,
but it can also indicate pain relief and self-healing. Cats
groom themselves meticulously u [...]"
Start of second doc: "Cat whiskers are highly sensitive tactile hairs that
help cats detect changes in their surroundings and navigate
in the dark. These whiskers are embed [...]"
Term-vectors of these documents: 
[[3. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0. 3. 0. 0. 0.]]
human readable versions:
[('cats', 3.0)]
[('cats', 2.0), ('whiskers', 3.0)]


# Finding nearest vectors

We can also easily find the nearest vector (among the rows of our matrix $M$) to a given query vector.

Remember that by computing the nearest vectors, we really want to find the most similar
documents (to our query vector).

In [13]:
def find_nearest_k(q, M, k):
  return sorted([(eucl_distance(q, v), i) for i,v in enumerate(M)])[:k]

#Task 2:

Play with the code below, by choosing different query term vectors.

* Is there anything that **bothers you**?
Hint: text at index 0 and one other have a special relationship!

* Can you think about any **undesirable behaviours**? Maybe we're **not doing things right**?!

* Any alternatives?

In [14]:
D = len(keywords)

# you can either tune the chosen coordinates of your query
query_term_vector = np.zeros(len(keywords)) # [0,0,0...]
query_term_vector[keywords.index("cats")] = 0 # how much is it about cats?
query_term_vector[keywords.index("dogs")] = 0 # ... and dogs
query_term_vector[keywords.index("trees")] = 1 # trees, etc.
query_term_vector[keywords.index("binary")] = 1

# Alternatively, we can also set it directly as:
# query_term_vector = [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.]

print(f"Your query term-vector as a vector in R^{D}: {query_term_vector}")

nearest = find_nearest_k(query_term_vector, M, 3)

for i, (d, ind) in enumerate(nearest):
  print(f'''#{i+1} nearest doc at dist {d : .4} and index {ind}
    term-vector:{M[ind]}
    human-readable: {get_human_readable_term_vector(M[ind], keywords)}
    snippet: "{docs[ind][:100]}..."\n''')

Your query term-vector as a vector in R^10: [0. 0. 1. 0. 1. 0. 0. 0. 0. 0.]
#1 nearest doc at dist  1.0 and index 12
    term-vector:[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
    human-readable: [('trees', 1.0)]
    snippet: "The growth rings of trees, visible in cross-sections
of trunks, hold records of environmental change..."

#2 nearest doc at dist  1.414 and index 2
    term-vector:[0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
    human-readable: [('trees', 1.0), ('photosynthesis', 1.0)]
    snippet: "Deciduous trees shed their leaves in the fall as an
adaptation to conserve water and energy during c..."

#3 nearest doc at dist  1.414 and index 9
    term-vector:[0. 0. 2. 0. 0. 0. 0. 0. 0. 0.]
    human-readable: [('trees', 2.0)]
    snippet: "A spanning tree of a graph is a subgraph that includes
all the vertices and is also a tree, crucial ..."

